# Tools and Chains

Inspiration from: https://www.youtube.com/watch?v=hI2BY7yl_Ac

Side note: It might be a fun project to see if we can extract information directly from the youtube video and finalize that output into code

In [25]:
import os
from langchain.prompts  import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [ ]:
# Make sure you have setup your environmental variables beforehand
os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_ORGANIZATION"]

### Basic LLM Chain: Testing fact extraction

Start by loading in model of choice

In [27]:
llm_openai = OpenAI(model_name="gpt-4-0613", temperature=0, max_tokens=512)

Let's feed an article to extract facts from, this is the free completely unstructured text

In [28]:
article = """
Paris Saint-Germain soccer superstar Neymar has reportedly agreed a two-year deal to join Saudi Arabian club Al-Hilal.

France’s leading sports daily L’Equipe said on its website Sunday that the PSG and Brazil forward will receive a total of 160 million euros ($175 million) over two seasons. No details were given as to how much the transfer fee would be for Neymar, who has also been linked with a return to Barcelona.

The Associated Press has asked defending French champion PSG to confirm the deal. The request was not immediately answered.

Neymar missed PSG’s season-opening 0-0 draw against Lorient in the French league on Saturday after training alone on Friday, with the club saying it was because he was recovering from a viral infection. But Neymar is heading for a PSG exit and could leave during the summer transfer window.

The 31-year-old Brazilian joined PSG from Barcelona for a world-record fee of 222 million euros (now $244 million) six years ago, the same year Kylian Mbappe joined from Monaco in a deal worth 180 million euros.

PSG had already received a world-record $332 million bid from Al-Hilal for Mbappe, who is in the last year of his contract and has been mired in a transfer standoff. Mbappe, who reportedly refused to meet with representatives from the Saudi club when they were in Paris last month, has been strongly linked with a move to Real Madrid next season on a free transfer.

Tensions between Mbappe and PSG eased a little on Sunday after he was allowed to return to training following “constructive and positive talks” between the two parties.

Mbappe sat out Saturday’s draw with Lorient after being left out of training. Midfielder Marco Verratti was also not selected by new coach Luis Enrique, and has been linked with a move to the Saudi league after joining PSG in 2012.

"""

Article is pretty long, around 1,800 characters

In [29]:
len(article)

1823

Example of general PromptTemplate usage

In [30]:
template = "Extract the key facts about this piece of text. Don't include opinions. Give each fact a number and keep short sentences:\n {text_input}"

fact_extraction_prompt = PromptTemplate(input_variables=["text_input"],
                                        template=template)
print(fact_extraction_prompt.format(text_input=article))

Extract the key facts about this piece of text. Don't include opinions. Give each fact a number and keep short sentences:
 
Paris Saint-Germain soccer superstar Neymar has reportedly agreed a two-year deal to join Saudi Arabian club Al-Hilal.

France’s leading sports daily L’Equipe said on its website Sunday that the PSG and Brazil forward will receive a total of 160 million euros ($175 million) over two seasons. No details were given as to how much the transfer fee would be for Neymar, who has also been linked with a return to Barcelona.

The Associated Press has asked defending French champion PSG to confirm the deal. The request was not immediately answered.

Neymar missed PSG’s season-opening 0-0 draw against Lorient in the French league on Saturday after training alone on Friday, with the club saying it was because he was recovering from a viral infection. But Neymar is heading for a PSG exit and could leave during the summer transfer window.

The 31-year-old Brazilian joined PSG 

Now we have a template with the article we are interested on in the template. Let's go ahead to build the chain and input it

Notice how build the chain without the actual artcile input in the prompt, but just the prompt template with the instructions

In [31]:
fact_extraction_chain = LLMChain(llm=llm_openai, prompt=fact_extraction_prompt)

Now we run the extraction chain with the actual article in the input "text_input" that we indicated in the prompt template

In [34]:
facts = fact_extraction_chain.run(article)
print(facts)

1. Paris Saint-Germain (PSG) soccer player Neymar has reportedly agreed to a two-year deal with Saudi Arabian club Al-Hilal.
2. Neymar will receive a total of 160 million euros ($175 million) over two seasons.
3. No details were given about the transfer fee for Neymar.
4. Neymar has been linked with a return to Barcelona.
5. The Associated Press has asked PSG to confirm the deal.
6. Neymar missed PSG’s season-opening 0-0 draw against Lorient due to recovery from a viral infection.
7. Neymar joined PSG from Barcelona for a world-record fee of 222 million euros ($244 million) six years ago.
8. PSG received a world-record $332 million bid from Al-Hilal for Kylian Mbappe.
9. Mbappe is in the last year of his contract and has been involved in a transfer standoff.
10. Mbappe has been strongly linked with a move to Real Madrid next season on a free transfer.
11. Mbappe returned to training following “constructive and positive talks” with PSG.
12. Marco Verratti was not selected by new coach L

### Building a Chain manually passing sequential LLM outputs

We are going to write an additional LLM that is instructed in the PromptTemplate to take a list of facts (i.e. the one we produced before) and use it to write a paragraph to be used to investors in the MLS

In [35]:
investor_update_prompt = PromptTemplate(
    input_variables=["facts"],
    template="You are an MLS Soccer Analyst. Take the following list of facts and use them to write a short paragraph for potential ivestors in the league. Don't leave out key info:\n\n {facts}"
)

We feed in the prompt into a new chain and run it with the facts extracted in the previous LLMChain

In [36]:
investor_update_chain = LLMChain(llm=llm_openai, prompt=investor_update_prompt)

investor_update = investor_update_chain.run(facts)

print(len(investor_update))
print(investor_update)

1346
Paris Saint-Germain (PSG) soccer star Neymar has reportedly agreed to a two-year deal with Saudi Arabian club Al-Hilal, a move that will see him earn a total of 160 million euros ($175 million) over two seasons. This comes after Neymar's six-year stint with PSG, which he joined from Barcelona for a world-record fee of 222 million euros ($244 million). The details of the transfer fee for Neymar to Al-Hilal have not been disclosed. Neymar's move comes amidst speculation of a return to Barcelona and after missing PSG’s season-opening 0-0 draw against Lorient due to recovery from a viral infection. 

In addition to Neymar's move, PSG has received a world-record $332 million bid from Al-Hilal for Kylian Mbappe, who is in the last year of his contract and has been involved in a transfer standoff. Mbappe, who has been strongly linked with a move to Real Madrid next season on a free transfer, recently returned to training following “constructive and positive talks” with PSG. 

Furthermore

Let's show an example of what this would be like for generating something that might be more useful and practical to pass to code/output, like a knowledge graph formatted text

In [38]:
kg_prompt = PromptTemplate(
    input_variables=["facts"],
    template="Take this list of facts and turn them into triples for a knowledge graph:\n\n {facts}"
)

kg_chain = LLMChain(llm=llm_openai, prompt=kg_prompt)

print(kg_chain.run(facts))

1. ("Neymar", "agreed to a two-year deal with", "Al-Hilal")
2. ("Neymar", "will receive", "160 million euros over two seasons")
3. ("Neymar", "transfer fee", "unknown")
4. ("Neymar", "has been linked with a return to", "Barcelona")
5. ("The Associated Press", "asked", "PSG to confirm the deal")
6. ("Neymar", "missed", "PSG’s season-opening 0-0 draw against Lorient due to recovery from a viral infection")
7. ("Neymar", "joined", "PSG from Barcelona for a world-record fee of 222 million euros six years ago")
8. ("PSG", "received", "a world-record $332 million bid from Al-Hilal for Kylian Mbappe")
9. ("Kylian Mbappe", "is in", "the last year of his contract and has been involved in a transfer standoff")
10. ("Kylian Mbappe", "has been strongly linked with a move to", "Real Madrid next season on a free transfer")
11. ("Kylian Mbappe", "returned to", "training following constructive and positive talks with PSG")
12. ("Marco Verratti", "has been linked with a move to", "the Saudi league")
13

### Now let's actually Chain all of these LLMs together

In [40]:
from langchain.chains import SimpleSequentialChain, SequentialChain

We'll put both designed chains above in the same Sequential Chain and feed in the original input

In [41]:
full_chain = SimpleSequentialChain(chains=[fact_extraction_chain, investor_update_chain], verbose=True)

response = full_chain.run(article)



> Entering new SimpleSequentialChain chain...
1. Paris Saint-Germain (PSG) soccer player Neymar has reportedly agreed to a two-year deal with Saudi Arabian club Al-Hilal.
2. Neymar will receive a total of 160 million euros ($175 million) over two seasons.
3. No details were given about the transfer fee for Neymar.
4. Neymar has been linked with a return to Barcelona.
5. The Associated Press has asked PSG to confirm the deal.
6. Neymar missed PSG’s season-opening 0-0 draw against Lorient due to recovery from a viral infection.
7. Neymar joined PSG from Barcelona for a world-record fee of 222 million euros ($244 million) six years ago.
8. PSG received a world-record $332 million bid from Al-Hilal for Kylian Mbappe.
9. Mbappe is in the last year of his contract and has been involved in a transfer standoff.
10. Mbappe has been strongly linked with a move to Real Madrid next season on a free transfer.
11. Tensions between Mbappe and PSG eased after he was allowed to return to training.
12

We could actually see process and middle inputs/outputs above. The actual results stored in the variable will be the final chain response

In [42]:
print(response)

Paris Saint-Germain (PSG) is experiencing significant player movement, which could potentially impact the Major League Soccer (MLS) market. Neymar, a key player for PSG, has reportedly agreed to a two-year deal with Saudi Arabian club Al-Hilal, worth a total of 160 million euros ($175 million). The transfer fee details remain undisclosed. Neymar's move comes after a period of speculation about a potential return to Barcelona and his absence from PSG's season-opening draw due to a viral infection. 

In addition to Neymar, PSG is also dealing with a potential transfer of Kylian Mbappe, who has been involved in a standoff over his contract, which is in its final year. Al-Hilal has made a world-record bid of $332 million for Mbappe, who has been strongly linked with a move to Real Madrid next season on a free transfer. 

Furthermore, midfielder Marco Verratti, who joined PSG in 2012, has been linked with a move to the Saudi league. These developments could potentially open up opportunities